In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../../DATASETS_TRACTATS/TOT.csv")

In [3]:
df = df.astype({"CAMPAÑA": str, "ID_FINCA": str, "ID_ZONA": str, "ID_ESTACION": str, "VARIEDAD": str, "MODO": str, "TIPO": str, "COLOR": str})

In [4]:
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder
encoder = ce.OrdinalEncoder(cols=["ID_FINCA", "ID_ZONA", "ID_ESTACION", "ALTITUD", "VARIEDAD", "COLOR", "TIPO", "MODO"])
df = encoder.fit_transform(df)

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [5]:
df = df[df["CAMPAÑA"] != "22"].drop(columns ="P/S")

In [6]:
X = df.drop(columns = ["PRODUCCION", "CAMPAÑA"])
y = df["PRODUCCION"]

In [7]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

def grid(X,y):
    gsc = GridSearchCV(
                estimator=xgb.XGBRegressor(),
                param_grid={"n_estimators": [300, 500, 1000],
                            "learning_rate": [0.15],
                            "max_depth": [40, 60, 100],
                            "min_child_weight": [1, 2],
                            "gamma":[0.0],
                            "colsample_bytree":[0.4],},
                cv=5, scoring=["r2", "neg_root_mean_squared_error"], refit = "r2", verbose=0, n_jobs=-1)
    gsc.fit(X,y)
    
    return gsc

#### Random

In [8]:
model_random = grid(X,y)

#### Testejant un any

In [ ]:
years_to_test = ["19", "20", "21"]
model_one_year_test = {}
for year in years_to_test:
    X_train = X[df["CAMPAÑA"] != year]
    y_train = y[df["CAMPAÑA"] != year]
    X_test = X[df["CAMPAÑA"] == year]
    y_test = y[df["CAMPAÑA"] == year]
    m_y = grid(X_train, y_train)
    score_y = m_y.score(X_test, y_test)
    model_one_year_test[year] = (m_y, score_y)

#### Per anys

In [ ]:
years_to_test2 = df["CAMPAÑA"].unique()
model_one_year = {}
for year in years_to_test2:
    X2 = X[df["CAMPAÑA"] == year]
    y2 = y[df["CAMPAÑA"] == year]
    m_y = grid(X2, y2)
    model_one_year[year] = m_y